In [1]:
import numpy as np

In [2]:
import onnx

In [3]:
import os

In [4]:
import glob

In [5]:
from onnx import numpy_helper

In [6]:
import tvm

In [7]:
from tvm import te

In [8]:
import tvm.relay as relay

In [9]:
mnist_model = onnx.load('../mnist/mnist-8.onnx')

In [10]:
# load module param and mod

In [11]:
input_name = "Input3"

In [12]:
shape_dict = {input_name: (1, 1, 28, 28)}
print(shape_dict)

{'Input3': (1, 1, 28, 28)}


In [13]:
mod, params = relay.frontend.from_onnx(mnist_model, shape_dict)

In [14]:
# compile

In [15]:
#print(mod)

In [16]:
# pc test first

In [17]:
with relay.build_config(opt_level=3):
    mod_lib= relay.build(mod, tvm.target.create("llvm"), params=params, mod_name='mnist')

/home/zgh/Workspace/github/tvm/python/tvm/target/target.py:520: UserWarning: tvm.target.create() is being deprecated. Please use tvm.target.Target() instead
  warnings.warn("tvm.target.create() is being deprecated. Please use tvm.target.Target() instead")
Cannot find config for target=llvm -keys=cpu -link-params=0, workload=('conv2d_NCHWc.x86', ('TENSOR', (1, 1, 32, 32), 'float32'), ('TENSOR', (8, 1, 5, 5), 'float32'), (1, 1), (0, 0, 0, 0), (1, 1), 'NCHW', 'NCHW', 'float32'). A fallback configuration is used, which may bring great performance regression.
Cannot find config for target=llvm -keys=cpu -link-params=0, workload=('conv2d_NCHWc.x86', ('TENSOR', (1, 8, 18, 18), 'float32'), ('TENSOR', (16, 8, 5, 5), 'float32'), (1, 1), (0, 0, 0, 0), (1, 1), 'NCHW', 'NCHW', 'float32'). A fallback configuration is used, which may bring great performance regression.
Cannot find config for target=llvm -keys=cpu -link-params=0, workload=('dense_pack.x86', ('TENSOR', (1, 256), 'float32'), ('TENSOR', 

In [18]:
print(mod_lib)

In [19]:
# lib export
lib_path = "../lib/mnist.so"

In [20]:
mod_lib.export_library(lib_path)

In [21]:
# json export

In [22]:
#graph_path = "./lib/mnist.json"

In [23]:
#with open(graph_path, 'w') as fo:
    #fo.write(graph)

In [23]:
# params export

In [24]:
param_path = "../lib/mnist.params"

In [25]:
with open(param_path, 'wb') as fo:
    fo.write(relay.save_param_dict(params))

In [80]:
# test module

In [26]:
test_data_dir = '../mnist/test_data_set_2'

In [27]:
inputs = []

In [28]:
inputs_num = len(glob.glob(os.path.join(test_data_dir, 'input_*.pb')))

In [29]:
for i in range(inputs_num):
    input_file = os.path.join(test_data_dir, 'input_{}.pb'.format(i))
    tensor = onnx.TensorProto()
    with open(input_file, 'rb') as f:
        tensor.ParseFromString(f.read())
    inputs.append(numpy_helper.to_array(tensor))

In [31]:
#loaded_json = open(graph_path).read()

In [32]:
loaded_lib = tvm.runtime.load_module(lib_path)

In [33]:
#loaded_param = tvm.runtime.load_param_dict(open(param_path, "rb").read())
loaded_param = bytearray(open(param_path, "rb").read())

In [34]:
# context

In [35]:
ctx = tvm.cpu()

In [36]:
module = tvm.contrib.graph_executor.GraphModule(loaded_lib["mnist"](ctx))

In [37]:
#module = tvm.contrib.graph_executor.create(loaded_json, loaded_lib, ctx)

In [38]:
module.load_params(loaded_param)

In [39]:
module.set_input("Input3", inputs[0])

# tes

In [40]:
module.run()

In [41]:
out_deploy = module.get_output(0).asnumpy()

In [42]:
print(out_deploy)

[[-2334.0884   -1113.6259    1076.5802    -860.23926   1588.5348
  -1534.3464   -2686.0781     -56.480286    74.574875  3715.3796  ]]
